In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
ps = SnowballStemmer('english')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aarya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake['content'] = fake['title']+fake['text']
fake['result'] = 0
fake = fake[['content' , 'result']]

In [4]:
true['content'] = true['title']+true['text']
true['result'] = 1
true = true[['content' , 'result']]

In [5]:
fake.shape

(23481, 2)

In [6]:
true.shape

(21417, 2)

In [7]:
dataset = pd.concat([fake , true])

In [8]:

dataset['con    tent'] = dataset['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [9]:
def stem(obj):
    y = []
    for i in obj.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [10]:
dataset['content'] = dataset['content'].apply(stem)

In [11]:
dataset = dataset.sample(frac=1 , random_state=0)

In [12]:
x = dataset['content']
y = dataset['result']

In [13]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=0)

In [14]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [15]:
regressor = LogisticRegression(random_state=0)
regressor.fit(x_train , y_train)

LogisticRegression(random_state=0)

In [16]:
y_train_predict = regressor.predict(x_train)
print(accuracy_score(y_train , y_train_predict))

0.9949811421613756


In [17]:
y_test_predict = regressor.predict(x_test)
print(accuracy_score(y_test , y_test_predict))
print(classification_report(y_test , y_test_predict))

0.9929621380846325
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5796
           1       0.99      0.99      0.99      5429

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [18]:
def model_test(news):
    testing_news = {"text":[news]}
    new_testing_news = pd.DataFrame(testing_news)
    new_testing_news["text"] = new_testing_news["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    new_testing_news["text"] = new_testing_news["text"].apply(stem)
    new_x_test = new_testing_news["text"]
    new_x_test = vectorizer.transform(new_x_test)
    pred_res = regressor.predict(new_x_test)
    if pred_res == 0:
        print("Fake News")
    else:
        print("True news")

In [23]:
news = str(input())
model_test(news)

True news
